In [4]:
import os
import pandas as pd
import calendar
import datetime
import monpa
from monpa import utils
import re
import numpy as np
from alphalens.tears import (create_returns_tear_sheet,
                      create_information_tear_sheet,
                      create_turnover_tear_sheet,
                      create_summary_tear_sheet,
                      create_full_tear_sheet,
                      create_event_returns_tear_sheet,
                      create_event_study_tear_sheet)

from alphalens.utils import get_clean_factor_and_forward_returns

os.chdir('/Users/arthur/Desktop/factor_investment')

/opt/anaconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [2]:
# 我們從stopwords_zh.txt這個檔案中匯入繁體中文的停用詞
with open('./202202_Intern/stopwords_zh.txt', 'r') as file:
    stopwords = file.read().splitlines()
file.close()

with open('./202202_Intern/NTUSD_negative_unicode.txt', 'r') as file:
    nega_words = file.read().splitlines()
file.close()

with open('./202202_Intern/NTUSD_positive_unicode.txt', 'r') as file:
    posi_words = file.read().splitlines()
file.close()

# 這個function用來將字串以  **正則化**  處理去除中文字元以外的字元
def clearSentence(sentence):
    return re.sub(r'[^\u4e00-\u9fa5]+', '', sentence)

def calSen(content):
    sentence_list = utils.short_sentence(content)
    tokenStr = []
    for sentence in sentence_list:
        sentence = clearSentence(sentence)
        tokens = monpa.cut(sentence)
        tokenStr += tokens
        
    senScore = 0
    for i in tokenStr:
        if i in posi_words:
            senScore += 1
        elif i in nega_words:
            senScore -= 1
    return senScore

def find_key(dic, value):
    for keys, values in dic.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
        if value == values:
            return(keys)
            break

# PPT 文章

In [3]:
path = '202202_Intern/PttData-Stock-5000.txt'
# path = '202202_Intern/PttData-Stock-500.txt'
# path = '202202_Intern/PttData-Stock-1000.txt'
# path = '202202_Intern/PttData-Gossiping-20220505111857.txt'

In [4]:
with open(path, 'r') as file:
#     text = []
#     for line in file.readlines():
#         text.append(line)
    text = file.read()
#     print(text)

text = pd.Series(text.split('\n'))

In [5]:
columns = ['時間', '作者', '看板', '標題', '內容']

In [6]:
text[text.str.contains('作者') & text.str.contains('看板')][0]

'作者justforsing (雯晴啦不是晴雯)看板Stock標題[閒聊] 2022/05/05盤中閒聊時間Thu May  5 08:30:00 2022'

In [7]:
test = text[0]
before_keyword, keyword, after_keyword = test.partition('看板')
print(before_keyword.partition('作者')[2])
print(after_keyword.partition('標題')[0])
topic = after_keyword.partition('標題')[2]
print(topic)
topic = topic.split(' ')
print(datetime.datetime(int(topic[-1]), datetime.datetime.strptime(topic[-5], '%b').month, int(topic[-3])))

justforsing (雯晴啦不是晴雯)
Stock
[閒聊] 2022/05/05盤中閒聊時間Thu May  5 08:30:00 2022
2022-05-05 00:00:00


In [8]:
ptt_stock = pd.DataFrame(columns = columns)
ptt_stock

if_add = False

content = ""
for i in text:

    if ('發信站' in i) | ('→' in i) | ('推 ' in i) | ('噓 ' in i):
        if_add = False

    if if_add == True:
        content = content+i
    elif (if_add == False) & (content != ""):
        ptt_stock.loc[ptt_stock.shape[0]-1, '內容'] = content
        content = ""
    
    if ('作者' in i) & ('看板' in i) & ('標題' in i):
#         print(i)
        if_add = True
        before_keyword, keyword, after_keyword = i.partition('看板')
        ptt_stock.loc[ptt_stock.shape[0], '作者'] = before_keyword.partition('作者')[2]
        ptt_stock.loc[ptt_stock.shape[0]-1, '看板'] = after_keyword.partition('標題')[0]
        ptt_stock.loc[ptt_stock.shape[0]-1, '標題'] = after_keyword.partition('標題')[2].partition('時間')[0]
        
        time = pd.Series( after_keyword.partition('時間')[2].split(' ') )
#         time = time[time != ''].reset_index(drop=True)
        time = [i for i in time if i != '']

        try:
            ptt_stock.loc[ptt_stock.shape[0]-1, '時間'] = datetime.date(int(time[-1]), datetime.datetime.strptime(time[-4], '%b').month, int(time[-3]))
            print(time[-1], time[-4], time[-3])
        except:
            try:
                ptt_stock.loc[ptt_stock.shape[0]-1, '時間'] = datetime.date(int(time[4]), datetime.datetime.strptime(time[1], '%b').month, int(time[2]))
                print(time[4], time[2], time[1])
            except:
                continue
        
ptt_stock['時間'] = pd.to_datetime(ptt_stock['時間']).dt.date

2022 May 5
2022 May 5
2022 May 5
2022 May 4
2022 May 4
2022 May 4
2022 May 4
2022 May 3
2022 May 3
2022 May 3
2022 May 2
2022 May 2
2022 May 2
2022 May 1
2022 May 1
2022 Apr 30
2022 Apr 30
2022 Apr 30
2022 Apr 29
2022 Apr 29
2022 Apr 29
2022 Apr 29
2022 Apr 28
2022 Apr 28
2022 Apr 28
2022 Apr 28
2022 Apr 28
2022 Apr 27
2022 Apr 27
2022 Apr 27
2022 Apr 27
2022 Apr 27
2022 Apr 26
2022 Apr 26
2022 Apr 26
2022 Apr 26
2022 Apr 26
2022 Apr 25
2022 Apr 25
2022 Apr 25
2022 Apr 25
2022 Apr 25
2022 Apr 24
2022 Apr 24
2022 Apr 23
2022 Apr 23
2022 Apr 23
2022 Apr 22
2022 Apr 22
2022 Apr 22
2022 Apr 22
2022 Apr 21
2022 Apr 21
2022 Apr 21
2022 Apr 21
2022 Apr 20
2022 Apr 20
2022 Apr 20
2022 Apr 20
2022 Apr 19
2022 Apr 19
2022 Apr 19
2022 Apr 18
2022 Apr 18
2022 Apr 18
2022 Apr 18
2022 Apr 17
2022 Apr 16
2022 Apr 16
2022 Apr 16
2022 Apr 15
2022 Apr 15
2022 Apr 15
2022 Apr 15
2022 Apr 14
2022 Apr 14
2022 Apr 14
2022 Apr 14
2022 Apr 13
2022 Apr 13
2022 Apr 13
2022 Apr 13
2022 Apr 12
2022 Apr 12
2022 Ap

2021 Oct 8
2021 Oct 8
2021 Oct 8
2021 Oct 8
2021 Oct 7
2021 Oct 7
2021 Oct 7
2021 Oct 7
2021 Oct 7
2021 Oct 6
2021 Oct 6
2021 Oct 6
2021 Oct 6
2021 Oct 6
2021 Oct 5
2021 Oct 5
2021 Oct 5
2021 Oct 5
2021 Oct 5
2021 Oct 5
2021 Oct 4
2021 Oct 4
2021 Oct 4
2021 Oct 4
2021 Oct 4
2021 Oct 4
2021 Oct 3
2021 Oct 3
2021 Oct 2
2021 Oct 2
2021 Oct 2
2021 Oct 1
2021 Oct 1
2021 Oct 1
2021 Oct 1
2021 Oct 1
2021 Sep 30
2021 Sep 30
2021 Sep 30
2021 Sep 30
2021 Sep 30
2021 Sep 29
2021 Sep 29
2021 Sep 29
2021 Sep 29
2021 Sep 28
2021 Sep 28
2021 Sep 28
2021 Sep 28
2021 Sep 27
2021 Sep 27
2021 Sep 27
2021 Sep 27
2021 Sep 27
2021 Sep 26
2021 Sep 26
2021 Sep 26
2021 Sep 25
2021 Sep 25
2021 Sep 24
2021 Sep 24
2021 Sep 24
2021 Sep 24
2021 Sep 23
2021 Sep 23
2021 Sep 23
2021 Sep 23
2021 Sep 23
2021 Sep 22
2021 Sep 22
2021 Sep 22
2021 Sep 22
2021 Sep 21
2021 Sep 21
2021 Sep 21
2021 Sep 20
2021 Sep 20
2021 Sep 20
2021 Sep 19
2021 Sep 18
2021 Sep 18
2021 Sep 17
2021 Sep 17
2021 Sep 17
2021 Sep 17
2021 Sep 16
2021

2021 Jun 15
2021 Jun 15
2021 Jun 15
2021 Jun 15
2021 Jun 14
2021 Jun 14
2021 Jun 14
2021 Jun 14
2021 Jun 14
2021 Jun 13
2021 Jun 13
2021 Jun 13
2021 Jun 12
2021 Jun 12
2021 Jun 12
2021 Jun 12
2021 Jun 11
2021 Jun 11
2021 Jun 11
2021 Jun 11
2021 Jun 11
2021 Jun 11
2021 Jun 11
2021 Jun 10
2021 Jun 10
2021 Jun 10
2021 Jun 10
2021 Jun 10
2021 Jun 10
2021 Jun 10
2021 Jun 10
2021 Jun 10
2021 Jun 10
2021 Jun 9
2021 Jun 9
2021 Jun 9
2021 Jun 9
2021 Jun 9
2021 Jun 9
2021 Jun 9
2021 Jun 9
2021 Jun 9
2021 Jun 8
2021 Jun 8
2021 Jun 8
2021 Jun 8
2021 Jun 8
2021 Jun 8
2021 Jun 8
2021 Jun 8
2021 Jun 8
2021 Jun 7
2021 Jun 7
2021 Jun 7
2021 Jun 7
2021 Jun 7
2021 Jun 7
2021 Jun 7
2021 Jun 7
2021 Jun 7
2021 Jun 6
2021 Jun 6
2021 Jun 6
2021 Jun 6
2021 Jun 6
2021 Jun 5
2021 Jun 5
2021 Jun 5
2021 Jun 5
2021 Jun 4
2021 Jun 4
2021 Jun 4
2021 Jun 4
2021 Jun 4
2021 Jun 4
2021 Jun 4
2021 Jun 4
2021 Jun 3
2021 Jun 3
2021 Jun 3
2021 Jun 3
2021 Jun 3
2021 Jun 3
2021 Jun 3
2021 Jun 3
2021 Jun 3
2021 Jun 3
2021 Jun 2

2021 Mar 5
2021 Mar 5
2021 Mar 5
2021 Mar 4
2021 Mar 4
2021 Mar 4
2021 Mar 4
2021 Mar 4
2021 Mar 3
2021 Mar 3
2021 Mar 3
2021 Mar 3
2021 Mar 3
2021 Mar 2
2021 Mar 2
2021 Mar 2
2021 Mar 2
2021 Mar 1
2021 Mar 1
2021 Mar 1
2021 Feb 28
2021 Feb 28
2021 Feb 27
2021 Feb 27
2021 Feb 27
2021 Feb 27
2021 Feb 26
2021 Feb 26
2021 Feb 26
2021 Feb 26
2021 Feb 26
2021 Feb 26
2021 Feb 26
2021 Feb 25
2021 Feb 25
2021 Feb 25
2021 Feb 25
2021 Feb 25
2021 Feb 25
2021 Feb 24
2021 Feb 24
2021 Feb 24
2021 Feb 24
2021 Feb 24
2021 Feb 23
2021 Feb 23
2021 Feb 23
2021 Feb 23
2021 Feb 23
2021 Feb 23
2021 Feb 22
2021 Feb 22
2021 Feb 22
2021 Feb 22
2021 Feb 22
2021 Feb 21
2021 Feb 21
2021 Feb 20
2021 Feb 20
2021 Feb 20
2021 Feb 20
2021 Feb 20
2021 Feb 19
2021 Feb 19
2021 Feb 19
2021 Feb 19
2021 Feb 19
2021 Feb 18
2021 Feb 18
2021 Feb 18
2021 Feb 18
2021 Feb 18
2021 Feb 17
2021 Feb 17
2021 Feb 17
2021 Feb 17
2021 Feb 17
2021 Feb 16
2021 Feb 16
2021 Feb 16
2021 Feb 16
2021 Feb 15
2021 Feb 15
2021 Feb 14
2021 Feb 13


2020 Jul 27
2020 Jul 27
2020 Jul 27
2020 Jul 27
2020 Jul 27
2020 Jul 27
2020 Jul 27
2020 Jul 26
2020 Jul 26
2020 Jul 26
2020 Jul 25
2020 Jul 25
2020 Jul 25
2020 Jul 24
2020 Jul 24
2020 Jul 24
2020 Jul 24
2020 Jul 24
2020 Jul 23
2020 Jul 23
2020 Jul 23
2020 Jul 23
2020 Jul 22
2020 Jul 22
2020 Jul 22
2020 Jul 22
2020 Jul 22
2020 Jul 22
2020 Jul 21
2020 Jul 21
2020 Jul 21
2020 Jul 21
2020 Jul 21
2020 Jul 21
2020 Jul 20
2020 Jul 20
2020 Jul 20
2020 Jul 20
2020 Jul 20
2020 Jul 20
2020 Jul 19
2020 Jul 19
2020 Jul 19
2020 Jul 19
2020 Jul 18
2020 Jul 18
2020 Jul 18
2020 Jul 18
2020 Jul 17
2020 Jul 17
2020 Jul 17
2020 Jul 17
2020 Jul 17
2020 Jul 16
2020 Jul 16
2020 Jul 16
2020 Jul 16
2020 Jul 16
2020 Jul 15
2020 Jul 15
2020 Jul 15
2020 Jul 15
2020 Jul 15
2020 Jul 15
2020 Jul 15
2020 Jul 14
2020 Jul 14
2020 Jul 14
2020 Jul 14
2020 Jul 14
2020 Jul 13
2020 Jul 13
2020 Jul 13
2020 Jul 13
2020 Jul 12
2020 Jul 12
2020 Jul 12
2020 Jul 11
2020 Jul 11
2020 Jul 11
2020 Jul 10
2020 Jul 10
2020 Jul 10
2020

2020 Feb 20
2020 Feb 20
2020 Feb 20
2020 Feb 19
2020 Feb 19
2020 Feb 19
2020 Feb 18
2020 Feb 18
2020 Feb 18
2020 Feb 17
2020 Feb 17
2020 Feb 17
2020 Feb 16
2020 Feb 15
2020 Feb 14
2020 Feb 14
2020 Feb 13
2020 Feb 13
2020 Feb 13
2020 Feb 12
2020 Feb 12
2020 Feb 11
2020 Feb 11
2020 Feb 11
2020 Feb 10
2020 Feb 10
2020 Feb 9
2020 Feb 9
2020 Feb 9
2020 Feb 8
2020 Feb 8
2020 Feb 7
2020 Feb 7
2020 Feb 7
2020 Feb 6
2020 Feb 6
2020 Feb 6
2020 Feb 6
2020 Feb 5
2020 Feb 5
2020 Feb 5
2020 Feb 4
2020 Feb 4
2020 Feb 4
2020 Feb 4
2020 Feb 3
2020 Feb 3
2020 Feb 3
2020 Feb 3
2020 Feb 2
2020 Feb 2
2020 Feb 1
2020 Feb 1
2020 Feb 1
2020 Jan 31
2020 Jan 31
2020 Jan 31
2020 Jan 30
2020 Jan 30
2020 Jan 30
2020 Jan 30
2020 Jan 30
2020 Jan 29
2020 Jan 29
2020 Jan 28
2020 Jan 28
2020 Jan 27
2020 Jan 26
2020 Jan 25
2020 Jan 24
2020 Jan 24
2020 Jan 23
2020 Jan 23
2020 Jan 22
2020 Jan 21
2020 Jan 21
2020 Jan 20
2020 Jan 20
2020 Jan 19
2020 Jan 18
2020 Jan 17
2020 Jan 17
2020 Jan 16
2020 Jan 16
2020 Jan 15
2020 Jan

2019 Apr 18
2019 Apr 18
2019 Apr 18
2019 Apr 17
2019 Apr 17
2019 Apr 17
2019 Apr 16
2019 Apr 16
2019 Apr 16
2019 Apr 15
2019 Apr 15
2019 Apr 15
2019 Apr 14
2019 Apr 13
2019 Apr 13
2019 Apr 12
2019 Apr 12
2019 Apr 12
2019 Apr 11
2019 Apr 11
2019 Apr 11
2019 Apr 11
2019 Apr 10
2019 Apr 10
2019 Apr 9
2019 Apr 9
2019 Apr 9
2019 Apr 9
2019 Apr 8
2019 Apr 8
2019 Apr 8
2019 Apr 7
2019 Apr 7
2019 Apr 6
2019 Apr 5
2019 Apr 5
2019 Apr 4
2019 Apr 4
2019 Apr 3
2019 Apr 3
2019 Apr 3
2019 Apr 2
2019 Apr 2
2019 Apr 2
2019 Apr 1
2019 Apr 1
2019 Apr 1
2019 Mar 31
2019 Mar 30
2019 Mar 30
2019 Mar 29
2019 Mar 29
2019 Mar 29
2019 Mar 28
2019 Mar 28
2019 Mar 28
2019 Mar 27
2019 Mar 27
2019 Mar 27
2019 Mar 26
2019 Mar 26
2019 Mar 25
2019 Mar 25
2019 Mar 25
2019 Mar 24
2019 Mar 24
2019 Mar 24
2019 Mar 23
2019 Mar 23
2019 Mar 22
2019 Mar 22
2019 Mar 22
2019 Mar 22
2019 Mar 21
2019 Mar 21
2019 Mar 21
2019 Mar 21
2019 Mar 20
2019 Mar 20
2019 Mar 20
2019 Mar 20
2019 Mar 19
2019 Mar 19
2019 Mar 19
2019 Mar 18
201

In [9]:
candidatePoolDF = pd.read_excel('./計算溢酬/CTBCdata/10個因子原始資料.xlsx','125檔', dtype=str,index_col=0,header=1)
candidatePool=list(candidatePoolDF.index)
candidatePool=[str(x) for x in candidatePool]
candidatePool

['2330',
 '2317',
 '6505',
 '2412',
 '2454',
 '2882',
 '1301',
 '1303',
 '2881',
 '2002',
 '1326',
 '3045',
 '2308',
 '2886',
 '1216',
 '2357',
 '2382',
 '2105',
 '2891',
 '4904',
 '2498',
 '3673',
 '2912',
 '3481',
 '1402',
 '2892',
 '2885',
 '2880',
 '2303',
 '5880',
 '1101',
 '2207',
 '2883',
 '2801',
 '1102',
 '2409',
 '2301',
 '2890',
 '4938',
 '3008',
 '2354',
 '2474',
 '9904',
 '2887',
 '2324',
 '2408',
 '2884',
 '2227',
 '2347',
 '2201',
 '2888',
 '3034',
 '5871',
 '2353',
 '3231',
 '2823',
 '1722',
 '2395',
 '1802',
 '9921',
 '2356',
 '9945',
 '1476',
 '2915',
 '2049',
 '5347',
 '1504',
 '3682',
 '4958',
 '9910',
 '2618',
 '2610',
 '2615',
 '1227',
 '2385',
 '4174',
 '8464',
 '4147',
 '2377',
 '6239',
 '2633',
 '3702',
 '3532',
 '6488',
 '6456',
 '2344',
 '2603',
 '8299',
 '3105',
 '2327',
 '1590',
 '2337',
 '3711',
 '2492',
 '2371',
 '5876',
 '2379',
 '2345',
 '2834',
 '6669',
 '6415',
 '5269',
 '4743',
 '3037',
 '8046',
 '2609',
 '8454',
 '4966',
 '3529',
 '8406',
 '6121',
 

In [10]:
stock = pd.read_csv('./data/2022_utf8.csv')
stock

/var/folders/w8/jwxq4stj29506b1pf67xpb000000gq/T/ipykernel_869/213076977.py:1: DtypeWarning: Columns (548) have mixed types. Specify dtype option on import or set low_memory=False.
  stock = pd.read_csv('./data/2022_utf8.csv')


,證券代碼,年月,季別,合併(Y/N),月份,幣別,現金及約當現金,透過損益按公允價值衡量之金融資產－流動,透過其他綜合損益按公允價值衡量之金融資產－流動,按攤銷後成本衡量之金融資產－流動,...,財報發布日,財報類別（1個別2個體3合併）,財報年月起日,財報年月迄日,市場別,交易所主產業代碼,交易所子產業代碼,TEJ主產業代碼,TEJ子產業代碼,財報附註TEJ是否完成Y/N
0,1101 台泥,2021/12,4,Y,12,NTD,91121942.0,363813.0,6387543.0,15508688.0,...,2022/02/25,3.0,2021/01,2021/12,TSE,M1100,M1100,11A,11A,Y
1,1110 東泥,2021/12,4,Y,12,NTD,419504.0,248685.0,0.0,0.0,...,2022/03/18,3.0,2021/01,2021/12,TSE,M1100,M1100,11A,11A,N
2,1216 統一,2021/12,4,Y,12,NTD,80837880.0,10394193.0,0.0,15300236.0,...,2022/03/09,3.0,2021/01,2021/12,TSE,M1200,M1200,12C,12C,Y
3,1229 聯華,2021/12,4,Y,12,NTD,2785515.0,103474.0,666216.0,0.0,...,2022/03/15,3.0,2021/01,2021/12,TSE,M1200,M1200,12B,12B,Y
4,1231 聯華食,2021/12,4,Y,12,NTD,327180.0,297223.0,0.0,0.0,...,2022/03/10,3.0,2021/01,2021/12,TSE,M1200,M1200,12Z,12Z,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120968,9985 觀天下,2005/06,2,N,6,NTD,91212.0,0.0,0.0,0.0,...,2006/08/22,1.0,2005/01,2005/06,NaN,M9900,M9900,99G,99G,Y
120969,9986 南天有線,2005/06,2,N,6,NTD,49042.0,0.0,0.0,0.0,...,2006/08/31,1.0,2005/01,2005/06,NaN,M9900,M9900,99G,99G,Y
120970,9989 天外天興業,2005/06,2,N,6,NTD,111207.0,235733.0,0.0,0.0,...,2006/08/30,1.0,2005/01,2005/06,NaN,M9900,M9900,99G,99G,Y
120971,A00010 富邦投信,2005/06,2,N,6,NTD,1050144.0,454900.0,0.0,0.0,...,2006/10/16,1.0,2005/01,2005/06,NaN,M2800,M3100,31A,31A,Y


In [11]:
coid_dic = {}

coid_dic['2801'] = '彰銀'
coid_dic['2823'] = '中壽'
coid_dic['2834'] = '臺企金'
coid_dic['2880'] = '華南金'
coid_dic['2881'] = '富邦金'
coid_dic['2882'] = '國泰金'
coid_dic['2883'] = '開發金'
coid_dic['2884'] = '玉山金'
coid_dic['2885'] = '元大金'
coid_dic['2886'] = '兆豐金'
coid_dic['2887'] = '台新金'
coid_dic['2888'] = '新光金'
coid_dic['2890'] = '永豐金'
coid_dic['2891'] = '中信金'
coid_dic['2892'] = '第一金'
coid_dic['5880'] = '合作金'
coid_dic['5876'] = '上海商銀'

for i in stock['證券代碼']:
    coid = i.split(' ')[0]
    name = i.split(' ')[1]
    coid_dic[coid] = name
coid_dic

{'2801': '彰銀',
 '2823': '中壽',
 '2834': '臺企金',
 '2880': '華南金',
 '2881': '富邦金',
 '2882': '國泰金',
 '2883': '開發金',
 '2884': '玉山金',
 '2885': '元大金',
 '2886': '兆豐金',
 '2887': '台新金',
 '2888': '新光金',
 '2890': '永豐金',
 '2891': '中信金',
 '2892': '第一金',
 '5880': '合作金',
 '5876': '上海商銀',
 '1101': '台泥',
 '1110': '東泥',
 '1216': '統一',
 '1229': '聯華',
 '1231': '聯華食',
 '1232': '大統益',
 '1233': '天仁',
 '1236': '宏亞',
 '1246': '中聯油脂',
 '1258': '其祥-KY',
 '1259': '安心',
 '1268': '漢來美食',
 '1294': '漢田',
 '1301': '台塑',
 '1303': '南亞',
 '1304': '台聚',
 '1305': '華夏',
 '1308': '亞聚',
 '1309': '台達化',
 '1313': '聯成',
 '1314': '中石化',
 '1316': '上曜',
 '1323': '永裕',
 '1326': '台化',
 '1409': '新纖',
 '1414': '東和',
 '1417': '嘉裕',
 '1434': '福懋',
 '1437': '勤益控',
 '1438': '三地開發',
 '1440': '南紡',
 '1451': '年興',
 '1452': '宏益',
 '1455': '集盛',
 '1456': '怡華',
 '1476': '儒鴻',
 '1480': '東盟開發',
 '1504': '東元',
 '1513': '中興電',
 '1515': '力山',
 '1516': '川飛',
 '1524': '耿鼎',
 '1526': '日馳',
 '1528': '恩德',
 '1535': '中宇',
 '1536': '和大',
 '1558': '伸興',
 '1560'

In [12]:
print(find_key(coid_dic, '台積電'))

2330


In [102]:
# S = pd.DataFrame(columns=candidatePool, index=pd.date_range(start="2010-5-31", end="2019-12-31"))
# S = pd.DataFrame(columns=candidatePool, index=pd.date_range(start="2022-3-22", end="2022-5-6"))
S = pd.DataFrame(columns=candidatePool, index=pd.date_range(start="2019-3-4", end="2019-12-31"))
S.index = pd.to_datetime(S.index).date

In [103]:
for i in S.index:
    for j in S.columns:
#         print(i, j)
        j = coid_dic[j]
#         print(i, j)
        news = ptt_stock[ptt_stock['時間'] == i]
        news = news[news.內容.str.contains(j, na=False)]
        news = news['內容']
#         print(i, j)
#         print(news)
        if len(news) == 0:
            continue
        else:
            totalScore = 0
#             print(news)
            for k in news:
                totalScore += calSen(k)
            j = find_key(coid_dic, j)
            print(i, j)
            S.loc[i, j] = totalScore
#             input("Press Enter to continue...")

2019-03-04 2330
2019-03-04 1303
2019-03-04 5347
2019-03-04 4743
2019-03-05 5347
2019-03-06 2888
2019-03-06 2823
2019-03-06 4147
2019-03-07 2317
2019-03-07 1303
2019-03-07 2474
2019-03-07 2408
2019-03-07 2377
2019-03-07 2327
2019-03-07 2492
2019-03-08 5347
2019-03-09 2382
2019-03-09 9921
2019-03-09 2345
2019-03-10 2317
2019-03-12 2888
2019-03-12 2377
2019-03-12 2327
2019-03-12 2371
2019-03-12 4743
2019-03-16 2324
2019-03-16 5347
2019-03-17 2317
2019-03-17 5347
2019-03-18 2330
2019-03-18 2317
2019-03-18 3008
2019-03-19 2882
2019-03-19 2881
2019-03-19 2886
2019-03-19 2891
2019-03-19 2498
2019-03-19 3673
2019-03-19 2885
2019-03-19 2303
2019-03-19 1101
2019-03-19 2883
2019-03-19 1102
2019-03-19 2409
2019-03-19 2890
2019-03-19 2884
2019-03-19 2888
2019-03-19 3231
2019-03-19 2356
2019-03-19 2610
2019-03-19 2633
2019-03-19 2337
2019-03-19 3711
2019-03-19 3037
2019-03-20 2330
2019-03-20 5347
2019-03-20 3105
2019-03-22 1216
2019-03-22 2912
2019-03-22 2303
2019-03-22 3037
2019-03-22 5904
2019-03-

2019-07-31 8299
2019-08-01 2801
2019-08-01 2371
2019-08-02 2330
2019-08-02 2317
2019-08-02 2327
2019-08-04 5347
2019-08-04 2492
2019-08-05 5347
2019-08-08 1216
2019-08-11 3045
2019-08-14 5347
2019-08-14 4147
2019-08-14 6488
2019-08-14 8299
2019-08-14 3105
2019-08-14 6147
2019-08-14 8069
2019-08-14 8436
2019-08-15 2382
2019-08-15 2324
2019-08-15 2356
2019-08-18 5347
2019-08-20 3481
2019-08-22 8436
2019-08-23 2330
2019-08-23 2409
2019-08-24 1303
2019-08-24 5347
2019-08-25 2308
2019-08-25 2382
2019-08-25 2327
2019-08-26 5347
2019-08-27 2317
2019-08-27 2303
2019-08-27 8436
2019-08-28 2882
2019-08-28 2881
2019-08-28 2356
2019-08-28 3702
2019-08-29 2801
2019-08-31 2317
2019-08-31 1303
2019-08-31 5880
2019-08-31 4938
2019-08-31 3034
2019-08-31 3231
2019-08-31 5347
2019-09-03 2330
2019-09-03 3481
2019-09-03 2409
2019-09-03 2337
2019-09-03 3037
2019-09-04 2454
2019-09-04 5347
2019-09-05 2330
2019-09-05 2454
2019-09-05 2357
2019-09-05 5347
2019-09-07 5347
2019-09-09 2330
2019-09-09 2409
2019-09-

In [104]:
S = S.fillna(method='ffill')
S = S.reindex(columns=S.columns.sort_values())

In [105]:
S

,1101,1102,1216,1227,1301,1303,1326,1402,1476,1504,...,8069,8299,8406,8436,8454,8464,9904,9910,9921,9945
2019-03-04,NaN,NaN,NaN,NaN,NaN,193,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-05,NaN,NaN,NaN,NaN,NaN,193,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-06,NaN,NaN,NaN,NaN,NaN,193,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-07,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-08,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,1.0,-1.0,15.0,-4.0,6.0,3,1.0,1.0,-3.0,0.0,...,0.0,0.0,NaN,0.0,NaN,NaN,0.0,-3.0,0.0,NaN
2019-12-28,1.0,-1.0,15.0,-4.0,6.0,3,1.0,1.0,-3.0,0.0,...,0.0,0.0,NaN,0.0,NaN,NaN,0.0,-3.0,0.0,NaN
2019-12-29,1.0,-1.0,15.0,-4.0,6.0,3,1.0,1.0,-3.0,0.0,...,0.0,0.0,NaN,0.0,NaN,NaN,0.0,-3.0,0.0,NaN
2019-12-30,1.0,-1.0,15.0,-4.0,6.0,3,1.0,1.0,-3.0,0.0,...,0.0,0.0,NaN,0.0,NaN,NaN,0.0,-3.0,0.0,NaN


In [31]:
S.columns.tolist()

['1101',
 '1102',
 '1216',
 '1227',
 '1301',
 '1303',
 '1326',
 '1402',
 '1476',
 '1504',
 '1565',
 '1590',
 '1722',
 '1802',
 '2002',
 '2049',
 '2105',
 '2201',
 '2207',
 '2227',
 '2301',
 '2303',
 '2308',
 '2317',
 '2324',
 '2327',
 '2330',
 '2337',
 '2344',
 '2345',
 '2347',
 '2353',
 '2354',
 '2356',
 '2357',
 '2371',
 '2377',
 '2379',
 '2382',
 '2385',
 '2395',
 '2408',
 '2409',
 '2412',
 '2454',
 '2474',
 '2492',
 '2498',
 '2603',
 '2609',
 '2610',
 '2615',
 '2618',
 '2633',
 '2801',
 '2823',
 '2834',
 '2880',
 '2881',
 '2882',
 '2883',
 '2884',
 '2885',
 '2886',
 '2887',
 '2888',
 '2890',
 '2891',
 '2892',
 '2912',
 '2915',
 '3008',
 '3034',
 '3037',
 '3045',
 '3105',
 '3176',
 '3231',
 '3293',
 '3481',
 '3529',
 '3532',
 '3552',
 '3673',
 '3682',
 '3702',
 '3711',
 '4147',
 '4157',
 '4174',
 '4743',
 '4904',
 '4938',
 '4958',
 '4966',
 '5269',
 '5347',
 '5483',
 '5530',
 '5871',
 '5876',
 '5880',
 '5903',
 '5904',
 '6121',
 '6147',
 '6239',
 '6415',
 '6446',
 '6456',
 '6488',
 

# 股價

In [35]:
price = pd.read_csv('./data/TEJ_UTF8.csv')
price['證券代碼'] = price['證券代碼'].astype('str')
price = price.loc[price.證券代碼.isin( S.columns ), ]
price = price.reset_index(drop=True)
price['年月日'] = pd.to_datetime(price['年月日']).dt.date

/var/folders/w8/jwxq4stj29506b1pf67xpb000000gq/T/ipykernel_869/3682079592.py:1: DtypeWarning: Columns (11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  price = pd.read_csv('./data/TEJ_UTF8.csv')


In [39]:
price = price.sort_values(['證券代碼', '年月日'])
price = price.reset_index(drop=True)
price

,證券代碼,簡稱,年月日,收盤價(元),成交量(千股),流通在外股數(千股),市值(百萬元),市值比重％,成交筆數(筆),本益比-TSE,本益比-TEJ,股價淨值比-TSE,股價淨值比-TEJ,股價營收比-TEJ,股利殖利率-TSE,現金股利率
0,1101,台泥,2010-01-04,14.3304,8299,3292175,112921,0.534,2839,18.8500,18.5400,1.4700,1.4700,5.25,3.85,3.85
1,1101,台泥,2010-01-05,14.8318,48059,3292175,116872,0.553,13190,19.5100,19.1900,1.5200,1.5200,5.43,3.72,3.72
2,1101,台泥,2010-01-06,15.1451,36026,3292175,119341,0.557,9057,19.9200,19.6000,1.5500,1.5500,5.54,3.64,3.64
3,1101,台泥,2010-01-07,14.8526,14531,3292175,117036,0.552,4733,19.5300,19.2200,1.5200,1.5200,5.44,3.71,3.71
4,1101,台泥,2010-01-08,14.7900,10671,3292175,116542,0.547,3456,19.4500,19.1400,1.5200,1.5200,5.41,3.73,3.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280927,9945,潤泰新,2019-12-25,20.7998,2461,1003254,45347,0.125,1476,4.1700,4.1795,0.55,0.5512,2.7621,4.4200,4.4248
280928,9945,潤泰新,2019-12-26,20.7768,1126,1003254,45297,0.125,678,4.1600,4.1749,0.55,0.5506,2.7591,4.4300,4.4297
280929,9945,潤泰新,2019-12-27,20.8919,1190,1003254,45548,0.125,766,4.1800,4.1980,0.55,0.5536,2.7743,4.4100,4.4053
280930,9945,潤泰新,2019-12-30,20.9379,1982,1003254,45648,0.126,1177,4.1900,4.2073,0.55,0.5549,2.7804,4.4000,4.3956


In [99]:
prices = pd.DataFrame(columns=S.columns, index=pd.date_range(start="2019-3-4", end="2019-12-31"))
prices.index = pd.to_datetime(prices.index).date
for i in prices.index: # 3/1~12/31所有日期
    print(i)
    date_price = price.loc[price['年月日']==i, '收盤價(元)'] # 某天所有的價錢 index為price擷取的index
    try:
        prices.loc[i, ] = date_price.values
    except:
        try:
            for j in date_price.index: # 從這些index回去查找證券代碼：price.loc[j, '證券代碼']
#                 print(j)
                prices.loc[i, price.loc[j, '證券代碼']] = date_price[j]
        except:
            continue
prices = prices.fillna(method='ffill')

2019-03-04
2019-03-05
2019-03-06
2019-03-07
2019-03-08
2019-03-09
2019-03-10
2019-03-11
2019-03-12
2019-03-13
2019-03-14
2019-03-15
2019-03-16
2019-03-17
2019-03-18
2019-03-19
2019-03-20
2019-03-21
2019-03-22
2019-03-23
2019-03-24
2019-03-25
2019-03-26
2019-03-27
2019-03-28
2019-03-29
2019-03-30
2019-03-31
2019-04-01
2019-04-02
2019-04-03
2019-04-04
2019-04-05
2019-04-06
2019-04-07
2019-04-08
2019-04-09
2019-04-10
2019-04-11
2019-04-12
2019-04-13
2019-04-14
2019-04-15
2019-04-16
2019-04-17
2019-04-18
2019-04-19
2019-04-20
2019-04-21
2019-04-22
2019-04-23
2019-04-24
2019-04-25
2019-04-26
2019-04-27
2019-04-28
2019-04-29
2019-04-30
2019-05-01
2019-05-02
2019-05-03
2019-05-04
2019-05-05
2019-05-06
2019-05-07
2019-05-08
2019-05-09
2019-05-10
2019-05-11
2019-05-12
2019-05-13
2019-05-14
2019-05-15
2019-05-16
2019-05-17
2019-05-18
2019-05-19
2019-05-20
2019-05-21
2019-05-22
2019-05-23
2019-05-24
2019-05-25
2019-05-26
2019-05-27
2019-05-28
2019-05-29
2019-05-30
2019-05-31
2019-06-01
2019-06-02

In [100]:
prices

,1101,1102,1216,1227,1301,1303,1326,1402,1476,1504,...,8069,8299,8406,8436,8454,8464,9904,9910,9921,9945
2019-03-04,27.7838,31.3829,65.1916,47.6752,89.3859,66.5384,92.5843,25.7800,324.0221,17.2816,...,28.1409,240.5395,210.4094,373.6626,183.5467,240.8095,34.9500,134.8894,159.3356,21.2285
2019-03-05,27.6383,31.3418,65.2815,47.2354,89.8262,66.1863,92.5843,25.7800,324.9453,17.3263,...,28.1839,240.5395,208.2085,381.6639,183.9109,242.1752,35.0404,134.8894,158.3956,21.1626
2019-03-06,27.8201,31.5059,65.9109,47.4113,90.7068,66.7144,93.4536,25.9089,327.7147,17.3711,...,28.0548,236.6943,209.5290,376.0630,184.2751,249.4586,35.5377,137.3742,156.0455,21.6460
2019-03-07,27.6383,31.3008,65.7311,46.7956,92.9085,67.0664,94.3230,25.8659,327.7147,17.3711,...,28.0548,229.0039,210.8496,354.0593,185.7318,247.6378,35.0404,137.7291,157.9255,21.3164
2019-03-08,27.6383,31.1777,64.4722,45.7400,91.5875,66.5384,94.3230,25.5651,325.4068,17.5502,...,27.8397,227.7221,218.3328,359.2602,184.6393,239.8991,35.2212,135.2443,156.0455,21.0087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,36.4014,41.6267,69.2216,65.0990,93.6699,68.8207,82.0844,27.2255,384.4989,24.5064,...,28.7930,308.5856,191.9117,266.9619,220.2471,261.2104,37.2375,158.7250,205.6892,20.8919
2019-12-28,36.4014,41.6267,69.2216,65.0990,93.6699,68.8207,82.0844,27.2255,384.4989,24.5064,...,28.7930,308.5856,191.9117,266.9619,220.2471,261.2104,37.2375,158.7250,205.6892,20.8919
2019-12-29,36.4014,41.6267,69.2216,65.0990,93.6699,68.8207,82.0844,27.2255,384.4989,24.5064,...,28.7930,308.5856,191.9117,266.9619,220.2471,261.2104,37.2375,158.7250,205.6892,20.8919
2019-12-30,36.7380,42.0194,69.2216,65.0990,92.6498,68.8207,81.0711,27.1348,378.8026,24.4598,...,28.7006,304.1133,192.8170,263.6892,220.6243,262.1585,37.0025,155.9122,204.7303,20.9379


In [114]:
# S.index = S.index.astype('datetime64[ns]')
# prices.index = prices.index.astype('datetime64[ns]')

factor_data = get_clean_factor_and_forward_returns(
    S.stack(),
    prices,
#     quantiles=4,
    periods=(5, 10 ,20 , 30 ,60 ,90, 120))# , 240, 360, 480))
#    filter_zscore=None)

create_full_tear_sheet(factor_data) # , long_short=False, group_neutral=False, by_group=False)

/opt/anaconda3/lib/python3.9/site-packages/alphalens/utils.py:159: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series(index=x.index)


Dropped 74.9% entries from factor data: 47.2% in forward returns computation and 27.7% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).


MaxLossExceededError: max_loss (35.0%) exceeded 74.9%, consider increasing it.

In [7]:
np.nansum([5, np.nan])

5.0